In [1]:
import pandas as pd
import numpy as np

#NLTK library used for NLP techniques
import nltk

#string library used for removing punctuation and stopwords
import string

#re is used for Regex operations
import re

#showing how many characters in a line
pd.set_option('display.max_colwidth', 300)   

#reading the final dataset
data = pd.read_csv("/Users/aditiwalia/Downloads/Dataset - Final Sheet.tsv", sep='\t', header=None)

#columns of the data frame
data.columns = ['username', 'tweet', 'label', 'random']


In [2]:
stopwords = nltk.corpus.stopwords.words('english')


In [3]:
'''
remove_extras implemented to remove the following:
1.) The hyperlinks and URLs in the tweet
2.) RT or retweet tag
3.) HTML character codes like &amp

Parameters: a list of tokens
Returns: a string with the removed elements
'''
def remove_extras(tokens):
    text_noUrl = []
    for word in tokens:
        if "https" in word or "http" in word or word == "RT" or word == "amp":
            continue
        text_noUrl.append(word)
    return text_noUrl

'''
remove_punctAndAlphanumeric implemented to remove the punctuations and other alphanumeric characters in the text

Parameters: a string
Returns: a string with the removed elements
''' 
def remove_punctAndAlphanumeric(text):
    text_noExtra = "".join([char for char in text if char not in string.punctuation and (char in string.ascii_letters or char in char in string.whitespace)])
    return text_noExtra


'''
remove_stop implemented to remove the stopwords except "IS" because "IS" is short form for ISIS

Parameters: a list of tokens
Returns: a list of tokens with stopword tokens removed
'''
def remove_stop(tokens):
    tokens_noStop = [word for word in tokens if word not in stopwords]
    return tokens_noStop

'''
lemmatize implemented to lemmatize words/reduce each word to its basic lemma or root

Parameters: a list of tokens
Returns: a list of tokens with lemmatized words
'''
wn = nltk.WordNetLemmatizer()
def lemmatize(tokens):
    lemma_tokens = [wn.lemmatize(word) for word in tokens]
    return lemma_tokens

'''
cleanAndTokenize implemented for the cleaning, tokenizing and lemmatizing the tweets

Parameters: a string (tweets)
Returns: a list of tokens (cleaned and lemmatized)
'''
def cleanAndTokenize(text):
    cleanedText = remove_punctAndAlphanumeric(text)
    tokens = re.split('\W+',cleanedText)
    tokens_removedExtras = remove_extras(tokens)
    tokens_removedStop = remove_stop(tokens_removedExtras)
    tokens_lemmatized = lemmatize(tokens_removedStop)
    return tokens_lemmatized
    

In [4]:
data['preprocessed_tweet'] = data['tweet'].apply(lambda x: cleanAndTokenize(x.lower()))
cleaned_tokens = data['tweet'].apply(lambda x: cleanAndTokenize(x.lower())).values.tolist()
print(type(cleaned_tokens))
#data.head(20)

<class 'list'>


In [5]:
data['tweet_len'] = data['tweet'].apply(lambda x: len(x) - x.count(" "))
data.head(20)

,username,tweet,label,random,preprocessed_tweet,tweet_len
0,jxnatc,@RondaRousey Good luck tonight Rowdy! We're pulling for you! #NoToISIS #Prayers4Paris,0,0.166829,"[rondarousey, good, luck, tonight, rowdy, pulling, notoisis, prayersparis]",75
1,ashuneras,this is such a good question,0,0.791709,"[good, question]",23
2,CSULBASI,Have you ever wondered what major fits your extracurricular interests? Well now you can find out with one simple quiz! ‚†Ä ‚†Ä Take the quiz https://bit.ly/2XXpPGD,0,0.087267,"[ever, wondered, major, fit, extracurricular, interest, well, find, one, simple, quiz, take, quiz]",138
3,dqkidd,"#KSATFIESTA hello Leilana, Eli, Maxi, Aliyah, Carmen, Gabby, Ellie and paul watching the parade!!!!!! Quirogas &amp; Amayas Viva Fiesta!!!!",0,0.861475,"[ksatfiesta, hello, leilana, eli, maxi, aliyah, carmen, gabby, ellie, paul, watching, parade, quirogas, amayas, viva, fiesta]",121
4,Khen_Bee,"If you want to leave, take good care. Hope you have a lot of nice things to wear, but then a lot of nice things turn bad out there.",0,0.514142,"[want, leave, take, good, care, hope, lot, nice, thing, wear, lot, nice, thing, turn, bad]",103
5,douglvslee,Regina Hall was my first girl crush. her character in Malibu‚Äôs Most Wanted did something to my lil hormones.,0,0.237339,"[regina, hall, first, girl, crush, character, malibus, wanted, something, lil, hormone]",92
6,lilwolf___,how do girls take cute tongue out pics??? I look like I‚Äôm ab to slob on a knob when I do that shit,0,0.991494,"[girl, take, cute, tongue, pic, look, like, im, ab, slob, knob, shit]",78
7,thebabydoll__,So did he really pass or what ?,0,0.362523,"[really, pas, ]",24
8,abubakerdimshqi,from the Heart love and respect for all #MUJAHDIN in BILAD #ALSHAM Brothers&amp;Sisters you are our hope to lead us to right way #IS,1,0.041372,"[heart, love, respect, mujahdin, bilad, alsham, brothersampsisters, hope, lead, u, right, way]",109
9,robertmanoin,you misspelled RED omg,0,0.437904,"[misspelled, red, omg]",19


In [6]:
radical_keywords = ['KUFFUR', 'IS', 'ISLAMIC STATE', '#ILOVEISIS', 'KUFFARS', 'mujahiddeen',
                    'kuffur', 'KUFFAR', 'kuffars', 'kuffar', 'kafir', 'MUJAHIDEEN', 'KUFAR', 'KAFIR',
                    'KUFR', 'mujahideen', '#IS', 'kufar', 'kufr', 'mujahid', 'JIHAD', 'jihad', 'MUJAHID',
                    'MUJAHIDDEEN', '#ISIS', 'Islamic State', '#ILoveISIS', 'ISIL', 'allah', 'Allah', 'Assad', 'assad', 'PKK',
                    'YPG', '#AleppoIsBurning', 'Aleppo', 'martydom', 'Martyrdom']

def radicalWordPresence(text):
    for key in radical_keywords:
        if key in text:
            return 1
    return 0

In [7]:
data['rad_key'] = data['tweet'].apply(lambda x:radicalWordPresence(x))


In [8]:
data[data['label'] == 0][data['rad_key']==1].count()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


username              640
tweet                 640
label                 640
random                640
preprocessed_tweet    640
tweet_len             640
rad_key               640
dtype: int64

In [9]:
data[data['username']=='mobi_ayubi']['preprocessed_tweet']

48          [rt, jazrawisham, islamic, state, wilaya, alanbar, two, istishadis, denoted, armoured, vehicle, iraqi, regiment, hq, near, saudi, border]
243                                                                            [rt, asgacu, thousand, raped, tortured, assads, jail, bah, give, crap]
309                                                     [rt, abduhark, anyone, tell, boy, assad, hate, may, react, striking, assad, nuclear, missile]
364                             [rt, jazrawisaraqib, alqaeda, ahrar, alshame, openly, fighting, islamic, state, u, airforce, two, dog, going, say, h]
369                                                      [rt, yaq, islamicstate, fighter, destroy, kilogram, narcotic, south, rafah, northern, sinai]
373                                                                                  [malcolmite, fsa, jaish, al, islam, trying, save, saa, damascus]
520                      [rt, malikabilarabi, umar, al, farooq, assassinated, kafir, may, allah, cur

In [11]:
import gensim
import gensim.models.keyedvectors as word2vec

#from gensim.models import Word2Vec, KeyedVectors

In [12]:
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

8

In [13]:
#model = Word2Vec.load("/Users/aditiwalia/Downloads/GoogleNews-vectors-negative300.bin.gz")
model = word2vec.KeyedVectors.load_word2vec_format('/Users/aditiwalia/Downloads/GoogleNews-vectors-negative300.bin.gz', binary=True)


In [14]:
def get_vector(processed_tweet):
        # convert to lowercase, ignore all special characters - keep only
        # alpha-numericals and spaces
        #sentence = re.sub(r'[^A-Za-z0-9\s]', r'', str(sentence).lower())

        vectors = [model.wv[w] for w in processed_tweet
                   if w in model.wv]

        v = np.zeros(model.vector_size)

        if (len(vectors) > 0):
            v = (np.array([np.sum(x) for x in zip(*vectors)])) / v.size

        return v

In [15]:
data['sentence_vec'] = data['preprocessed_tweet'].apply(lambda x: get_vector(x))
data.head(25)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


,username,tweet,label,random,preprocessed_tweet,tweet_len,rad_key,sentence_vec
0,jxnatc,@RondaRousey Good luck tonight Rowdy! We're pulling for you! #NoToISIS #Prayers4Paris,0,0.166829,"[rondarousey, good, luck, tonight, rowdy, pulling, notoisis, prayersparis]",75,1,"[-6.225586e-05, 0.0015152994, 0.0004683431, 0.00092041015, 0.00029947917, -0.0016518148, -0.0021651203, 0.0005985514, 0.0006939697, 0.0021299236, -0.00016764323, -0.0004992676, -0.000785319, 0.0008203125, -0.002565918, 0.0015596517, 0.0018098959, 0.0016503907, 0.00026041668, -0.0020345051, 0.001..."
1,ashuneras,this is such a good question,0,0.791709,"[good, question]",23,0,"[0.0004720052, 0.000538737, -0.00018351237, 0.0011474609, -0.00032389324, 0.00037455242, 0.00025024413, 0.00046000164, 0.00082682294, 0.0010221354, -0.001184082, -0.0003486379, 0.00015625, 0.00014322916, -0.0015657552, 0.0018001302, 0.0014192709, 0.00028401692, -0.001484375, -0.00034830728, 0.00..."
2,CSULBASI,Have you ever wondered what major fits your extracurricular interests? Well now you can find out with one simple quiz! ‚†Ä ‚†Ä Take the quiz https://bit.ly/2XXpPGD,0,0.087267,"[ever, wondered, major, fit, extracurricular, interest, well, find, one, simple, quiz, take, quiz]",138,0,"[0.000399882, 0.00086273195, 0.00075050356, 0.004699707, -0.0026204428, 0.00018819173, 0.0057739764, -0.0003137207, 0.0011141014, -0.00018218994, 0.0020232138, -0.0036477407, -3.159841e-05, 0.0053710938, -0.0027394614, 0.005851847, 0.0020944213, 0.0014611817, 0.0018277995, -0.0016399638, -9.1451..."
3,dqkidd,"#KSATFIESTA hello Leilana, Eli, Maxi, Aliyah, Carmen, Gabby, Ellie and paul watching the parade!!!!!! Quirogas &amp; Amayas Viva Fiesta!!!!",0,0.861475,"[ksatfiesta, hello, leilana, eli, maxi, aliyah, carmen, gabby, ellie, paul, watching, parade, quirogas, amayas, viva, fiesta]",121,0,"[-0.0004971314, 0.0031913249, -0.00023061117, 0.009244791, -0.0012882487, -0.006554769, 0.0022745768, -0.0023860678, -0.001307373, 0.005003077, -0.0008247884, -0.006509603, -0.004166667, 0.0032674153, 0.0008396403, 0.005005697, 0.007045695, 0.0032552083, 0.0006599935, -0.004169922, 0.0019611614,..."
4,Khen_Bee,"If you want to leave, take good care. Hope you have a lot of nice things to wear, but then a lot of nice things turn bad out there.",0,0.514142,"[want, leave, take, good, care, hope, lot, nice, thing, wear, lot, nice, thing, turn, bad]",103,0,"[0.0030818684, 0.00426473, -0.00021728515, 0.0075638834, -0.002969157, -0.001755778, 0.0068082684, -0.004045817, 0.0007737223, 0.0043066405, -0.0014591471, -0.0069466145, 0.0013545735, 0.0011352539, -0.0063069663, 0.006693522, 0.004665934, 0.0074277734, -0.002055664, -0.0054378253, -0.001104126,..."
5,douglvslee,Regina Hall was my first girl crush. her character in Malibu‚Äôs Most Wanted did something to my lil hormones.,0,0.237339,"[regina, hall, first, girl, crush, character, malibus, wanted, something, lil, hormone]",92,0,"[0.0016145833, 0.0025083923, 0.0011762492, -0.00032867433, -0.0012475586, -0.00022054036, -0.00015625, -0.003592936, 9.7554526e-05, 0.0016183726, -0.00041707358, -0.0058211265, 0.0005008952, -0.0024979655, -0.0027505367, 0.0025853475, -0.0007423909, 0.0031634776, 0.002368164, -0.0023966471, 0.00..."
6,lilwolf___,how do girls take cute tongue out pics??? I look like I‚Äôm ab to slob on a knob when I do that shit,0,0.991494,"[girl, take, cute, tongue, pic, look, like, im, ab, slob, knob, shit]",78,0,"[0.002492574, 0.0011839294, 0.00080998737, 0.004847819, -0.0048441538, 0.00226532, 0.0025923664, -0.003221639, 0.0017877198, 0.002425944, -0.0028759765, -0.007281901, -0.0021687825, 0.00059082033, -0.0038867188, 0.0032760366, -0.0019381713, 0.0062333425, -0.0018850708, -0.002217814, -0.000811691..."
7,thebabydoll__,So did he really pass or what ?,0,0.362523,"[really, pas, ]",24,0,"[0.00081542967, -0.00024373372, 7.8125e-05, 0.0012369792, -0.0004671224, 0.00016601563, 0.00034912108, -0.00032063804, 0.0002882894, 0.00030354818, -0.0011409505, -0.0018619791, -0.001

In [18]:
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(data[['sentence_vec', 'tweet_len', 'rad_key']], data['label'], test_size=0.2)

X=data[['sentence_vec', 'tweet_len', 'rad_key']]
y=data['label']


#Using Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
#X_train_vect = pd.concat([X_train[['rad_key','tweet_len']].reset_index(drop=True),pd.DataFrame(X_train['sentence_vec'].values.tolist())], axis=1)
#X_test_vect = pd.concat([X_test[['rad_key','tweet_len']].reset_index(drop=True), X_test['sentence_vec'].apply(pd.Series)], axis=1)
#X_test_vect = pd.concat([X_test[['rad_key','tweet_len']].reset_index(drop=True), pd.DataFrame(X_test['sentence_vec'].to_numpy())], axis=1)
#X_test_vect = pd.concat([X_test[['rad_key','tweet_len']].reset_index(drop=True), pd.DataFrame(X_test['sentence_vec'].values.tolist())], axis=1)

X_train_vect=pd.concat([X[['rad_key','tweet_len']].reset_index(drop=True), pd.DataFrame(X['sentence_vec'].values.tolist())], axis=1)
#X_vect.tail(40)
#y.head(40)

In [17]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [ ]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X_train_vect, y, cv=5)
print(scores)

In [19]:
#GridSearch SVM
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
#from sklearn.tree import DecisionTreeClassifier

#dt = DecisionTreeClassifier(max_depth=5, random_state=0)
sv = SVC()
param = {
    'kernel': ['linear','poly','rbf'],
    'C': [1,100,1000]
}

gs = GridSearchCV(sv, param, cv=5, n_jobs=-1)
cv_fit = gs.fit(X_train_vect,y)
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,4.519594,0.168714,0.631069,0.014725,1000,rbf,"{'C': 1000, 'kernel': 'rbf'}",0.872359,0.857394,0.870599,0.869604,0.867841,0.867559,0.005289,1
5,3.581973,0.074100,0.694070,0.022227,100,rbf,"{'C': 100, 'kernel': 'rbf'}",0.871479,0.858275,0.866197,0.869604,0.866079,0.866327,0.004521,2
3,415.530744,215.979679,0.463528,0.020894,100,linear,"{'C': 100, 'kernel': 'linear'}",0.860915,0.846831,0.860035,0.852863,0.859912,0.856111,0.005470,3
6,452.509622,158.912721,0.427908,0.070917,1000,linear,"{'C': 1000, 'kernel': 'linear'}",0.863556,0.845070,0.857394,0.849339,0.856388,0.854350,0.006471,4
0,244.628866,23.410814,0.673387,0.017069,1,linear,"{'C': 1, 'kernel': 'linear'}",0.844190,0.844190,0.844190,0.844934,0.844934,0.844488,0.000364,5


In [20]:
#Grid Search RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
#from sklearn.tree import DecisionTreeClassifier

#dt = DecisionTreeClassifier(max_depth=5, random_state=0)
rf = RandomForestClassifier()
param = {
    'n_estimators': [10,50,100,200],
    'max_depth': [None,10,20,30]
}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
cv_fit = gs.fit(X_train_vect,y)
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,7.117576,0.229126,0.036578,0.004184,None,100,"{'max_depth': None, 'n_estimators': 100}",0.911092,0.901408,0.917254,0.919824,0.907489,0.911413,0.006640,1
15,13.348028,1.028865,0.042978,0.005258,30,200,"{'max_depth': 30, 'n_estimators': 200}",0.916373,0.897887,0.916373,0.916300,0.903965,0.910180,0.007796,2
3,14.992494,0.168932,0.063759,0.001428,None,200,"{'max_depth': None, 'n_estimators': 200}",0.917254,0.894366,0.920775,0.915419,0.903084,0.910179,0.009901,3
11,16.500943,0.184843,0.080272,0.014438,20,200,"{'max_depth': 20, 'n_estimators': 200}",0.916373,0.891725,0.913732,0.918943,0.903084,0.908771,0.010087,4
7,11.946753,0.196112,0.061078,0.004841,10,200,"{'max_depth': 10, 'n_estimators': 200}",0.910211,0.890845,0.913732,0.915419,0.904846,0.907011,0.008856,5


In [21]:
#Grid Search AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=5, random_state=0)
rf = AdaBoostClassifier(base_estimator=dt)
param = {
    'n_estimators': [50,100,200],
    'learning_rate': [0.1,0.5,1]
}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
cv_fit = gs.fit(X_train_vect,y)
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,151.377902,7.646242,0.146859,0.014226,1,200,"{'learning_rate': 1, 'n_estimators': 200}",0.919894,0.902289,0.926056,0.925991,0.913656,0.917577,0.008913,1
5,183.397393,0.437327,0.216404,0.006544,0.5,200,"{'learning_rate': 0.5, 'n_estimators': 200}",0.925176,0.904930,0.923415,0.918943,0.913656,0.917224,0.007327,2
7,90.440303,2.787735,0.102619,0.006566,1,100,"{'learning_rate': 1, 'n_estimators': 100}",0.921655,0.903169,0.919014,0.902203,0.899559,0.909120,0.009270,3
2,195.760239,1.691984,0.208319,0.004484,0.1,200,"{'learning_rate': 0.1, 'n_estimators': 200}",0.914613,0.902289,0.917254,0.916300,0.891630,0.908417,0.009980,4
4,90.861235,0.579600,0.103955,0.002226,0.5,100,"{'learning_rate': 0.5, 'n_estimators': 100}",0.913732,0.896127,0.910211,0.898678,0.898678,0.903485,0.007080,5


In [22]:
#Grid Search KNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

kn = KNeighborsClassifier()
param = {
    'n_neighbors': [3,5,11,19],
    'weights': ['uniform','distance'],
    'metric':['euclidean','manhattan']
}

gs = GridSearchCV(kn, param, cv=5, n_jobs=-1)
cv_fit = gs.fit(X_train_vect,y)
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
15,0.097535,0.006593,0.088721,0.009170,manhattan,19,distance,"{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}",0.880282,0.877641,0.887324,0.886344,0.883700,0.883058,0.003646,1
14,0.111420,0.002247,0.155529,0.008567,manhattan,19,uniform,"{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'uniform'}",0.882042,0.873239,0.889965,0.881057,0.880176,0.881296,0.005328,2
6,0.107411,0.001919,0.181649,0.002753,euclidean,19,uniform,"{'metric': 'euclidean', 'n_neighbors': 19, 'weights': 'uniform'}",0.881162,0.871479,0.888204,0.879295,0.881057,0.880239,0.005340,3
3,0.108876,0.004811,0.120696,0.002699,euclidean,5,distance,"{'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}",0.875880,0.880282,0.885563,0.875771,0.876652,0.878830,0.003748,4
11,0.109585,0.003837,0.101567,0.005551,manhattan,5,distance,"{'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}",0.875000,0.879401,0.881162,0.876652,0.881057,0.878655,0.002449,5
